In [ ]:
import turtle
import numpy as np
def sin(x):
    return np.sin(x*2*np.pi/360)
def cos(x):
    return np.cos(x*2*np.pi/360)
    
class Link:
    def __init__(self, a, b, theta, L, color, size):
        self.start = turtle.Turtle()
        self.end = turtle.Turtle()
        
        # build properties
        self.length = L
        self.joint_x = a
        self.joint_y = b
        self.angle = theta
        self.end_x = a + self.length * cos(self.angle) 
        self.end_y = b + self.length * sin(self.angle)
        
        self.color = color
        self.size = size
        
        # draw the starting point
        self.start.ht()
        self.start.shape('circle')
        self.start.color((0,0,0))
        self.start.turtlesize(size*0.07)
        self.start.speed(0)
        self.start.penup()
        self.start.goto(self.joint_x, self.joint_y)
        self.start.st()
        
        # draw end point
        self.end.ht()
        self.end.shape('circle')
        self.end.color(color)
        self.end.turtlesize(size*0.05)
        self.end.speed(0)
        self.end.penup()
        self.end.goto(self.end_x, self.end_y)
        self.start.st()
        
        # Draw the connecting lines of the link
        turtle.tracer(n=2, delay=0) 
        self.pen = turtle.Turtle()
        self.pen.ht()
        self.pen.clear()
        self.pen.speed(0)
        self.pen.pencolor(color)
        self.pen.width(size)
        self.pen.penup()
        self.pen.goto(self.joint_x, self.joint_y)
        self.pen.pendown()
        self.pen.goto(self.end_x, self.end_y)
        self.pen.penup()
    def clear(self):
        self.pen.clear()
        self.start.ht()
        self.end.ht()
    def moveto(self, a, b, theta, penState=True):
        # if penState:
        #     self.end.pendown()
        # else:
        #     self.end.penup()
        self.clear()
        self.joint_x = a
        self.joint_y = b
        self.angle = theta
        self.end_x = a + self.length * cos(self.angle) 
        self.end_y = b + self.length * sin(self.angle)
        
        # draw the starting point
        self.start.ht()
        self.start.shape('circle')
        self.start.color((0,0,0))
        self.start.turtlesize(self.size*0.07)
        self.start.speed(0)
        self.start.penup()
        self.start.goto(self.joint_x, self.joint_y)
        self.start.st()
        
        # draw end point
        self.end.ht()
        self.end.shape('circle')
        self.end.color(self.color)
        self.end.turtlesize(self.size*0.05)
        self.end.speed(0)
        if penState:
            self.end.pendown()
        else:
            self.end.penup()
        self.end.goto(self.end_x, self.end_y)
        self.start.st()
        
        # Draw the connecting lines of the link
        turtle.tracer(n=2, delay=0) 
        self.pen = turtle.Turtle()
        self.pen.ht()
        self.pen.clear()
        self.pen.speed(0)
        self.pen.pencolor(self.color)
        self.pen.width(self.size)
        self.pen.penup()
        self.pen.goto(self.joint_x, self.joint_y)
        self.pen.pendown()
        self.pen.goto(self.end_x, self.end_y)
        self.pen.penup()
        

class RobotArm:
    def __init__(self, x, y, theta1, theta2, theta3, theta4, L1, L2, L3, L4): 
        self.L1 = L1
        self.L2 = L2
        self.L3 = L3
        self.L4 = L4
        self.theta1 = theta1
        self.theta2 = theta2
        self.theta3 = theta3
        self.theta4 = theta4
        self.link1 = Link(x, y, theta1, L1, 'black',8)
        self.link2 = Link(self.link1.end_x, self.link1.end_y, theta1+theta2, L2, 'blue',7)
        self.link3 = Link(self.link2.end_x, self.link2.end_y, theta1+theta2+theta3, L3, 'green',6)
        self.link4 = Link(self.link3.end_x, self.link3.end_y, theta1+theta2+theta3+theta4, L4, 'red',5)
    def moveto(self, theta1, theta2, theta3, theta4, penState=True):
        self.theta1 = theta1
        self.theta2 = theta2
        self.theta3 = theta3
        self.theta4 = theta4
        self.link1.moveto(self.link1.joint_x, self.link1.joint_y, theta1, False)
        self.link2.moveto(self.link1.end_x, self.link1.end_y, theta1+theta2, False)
        self.link3.moveto(self.link2.end_x, self.link2.end_y, theta1+theta2+theta3, False)
        self.link4.moveto(self.link3.end_x, self.link3.end_y, theta1+theta2+theta3+theta4, penState)
    def dot(self):
        self.link4.end.dot()    
    

class Particle:
    # Repeatability
    np.random.seed(0)
    def __init__(self, num_particles = 1000, Range = 240):
        # Generate (-Range/2)~(Range/2) particles。
        # Range: Angular range of the robotic arm。-120~120，Range is 240。
        self.num_particles = num_particles
        self.Range = Range
        self.particles = np.random.rand(num_particles, 4)
        self.particles = self.particles * np.array((Range, Range, Range, Range))
        self.particles = self.particles - (self.Range / 2)
        return None
    def forward(self, L1,L2,L3,L4,theta1,theta2,theta3,theta4):
        x = L1*cos(theta1) + L2*cos(theta1 + theta2) + L3*cos(theta1+theta2+theta3)+L4*cos(theta1+theta2+theta3+theta4)
        y = L1*sin(theta1) + L2*sin(theta1 + theta2) + L3*sin(theta1+theta2+theta3)+L4*sin(theta1+theta2+theta3+theta4)
        x=x-200
        y=y-200
        return [x,y]
    def importanceW(self, goal, joints):
        # goal(vector): target coordinates[x,y]
        errors = np.zeros((self.num_particles,1))
        # errors: error per particle

        locations = np.zeros((self.num_particles,2))
        # locations The terminal coordinates of the joint angle of each particle

        for i in range(self.num_particles):
            [x,y]=p.forward(100,100,100,100,self.particles[i][0],self.particles[i][1],self.particles[i][2],self.particles[i][3])
            locations[i][0] = x
            locations[i][1] = y
       
        for i in range(self.num_particles):
            errors[i][0] = (locations[i][0]-goal[0])**2 + (locations[i][1]-goal[1])**2
            errors[i][0] = errors[i][0] + (self.particles[i][0]-joints[0])**2
            errors[i][0] = errors[i][0] + (self.particles[i][1]-joints[1])**2
            errors[i][0] = errors[i][0] + (self.particles[i][2]-joints[2])**2
            errors[i][0] = errors[i][0] + (self.particles[i][3]-joints[3])**2
        
        return errors

    def compute_weights(self, errors):
        #weights = np.max(errors) - errors
        weights = 1/(1+errors)
        weights = weights**4
        
        return weights  

    def resample(self, weights):
        # Normalize to get valid PDF
        probabilities = weights / np.sum(weights)
        probabilities = probabilities[:,0] # Change the dimension to 1
        # Resample
        indices = np.random.choice(
            self.num_particles,
            size = self.num_particles,
            p = probabilities)
        self.particles = self.particles[ indices, : ]
        
        # Take average over all particles, best-guess for location
        th1 = np.mean(self.particles[:,0])
        th2 = np.mean(self.particles[:,1])
        th3 = np.mean(self.particles[:,2])
        th4 = np.mean(self.particles[:,3])
        
        return [th1,th2,th3,th4]
    def apply_noise(self):
        # Noise is good!  Noise expresses our uncertainty in the target's position and velocity
        # We add small variations to each hypothesis that were samples from the best ones in last iteration.
        # The target's position and velocity may have changed since the last frame, some of the fuzzed hypotheses will match these changes.
        SIGMA = 1.0
        
        noise = np.concatenate(
            (
                np.random.normal(0.0, SIGMA, (self.num_particles,1)),
                np.random.normal(0.0, SIGMA, (self.num_particles,1)),
                np.random.normal(0.0, SIGMA, (self.num_particles,1)),
                np.random.normal(0.0, SIGMA, (self.num_particles,1))
            ),
            axis=1
        )
        self.particles += noise    
    def batch(self, goal, joints):
        e = self.importanceW(goal, joints) 
        w = self.compute_weights(e)
        ang = self.resample(w)
        self.apply_noise()
        return ang
    

myarm = RobotArm(-200,-200, 20,-20,20,-20, 100,100,100,100)        

p = Particle(5000)
p.num_particles

x = 201
y = -201

for i in range(50):
    #current joints
    j1 = myarm.theta1
    j2 = myarm.theta2
    j3 = myarm.theta3
    j4 = myarm.theta4
    
    # goal
    x = x-1
    y = y+1
    #print(x,y)
    ang=p.batch([x,y],[j1,j2,j3,j4])
    #print(ang)
    myarm.moveto(ang[0],ang[1],ang[2],ang[3],False)
    
x = 151
y = -151    

for i in range(301):
    #current joints
    j1 = myarm.theta1
    j2 = myarm.theta2
    j3 = myarm.theta3
    j4 = myarm.theta4
    
    # goal
    x = x - 1
    y = y + 1 
    #print(x,y)
    ang=p.batch([x,y],[j1,j2,j3,j4])
    #print(ang)
    myarm.moveto(ang[0],ang[1],ang[2],ang[3],True)
    

turtle.done()
